<a href="https://colab.research.google.com/github/GlebSimon/Gleb/blob/main/Sql_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
100% 6.50M/6.50M [00:00<00:00, 85.7MB/s]


In [ ]:
import sqlite3

In [ ]:
import pandas as pd

In [ ]:
con = sqlite3.connect('db')

In [ ]:
sqlite3.sqlite_version

'3.38.0'

In [ ]:
def  select(sql):
  return pd.read_sql(sql, con)  

**% изменения выручки по клиентам за период**



In [ ]:
t= pd.DataFrame({'user_id':[1,1,1,2,2,2] ,'event_date':pd.to_datetime(['2021-04-01','2021-04-02','2021-04-03','2021-04-01','2021-04-02','2021-04-03'],
                                                              format='%Y-%m-%d'),'revenue':[1000,800,3000,2000,2500,2200]})

In [ ]:
t.to_sql('revenue',con,index=False,if_exists='replace')

In [ ]:
sql = '''select * from revenue'''

In [ ]:
cur = con.cursor()

In [ ]:
select(sql)

,user_id,event_date,revenue
0,1,2021-04-01 00:00:00,1000
1,1,2021-04-02 00:00:00,800
2,1,2021-04-03 00:00:00,3000
3,2,2021-04-01 00:00:00,2000
4,2,2021-04-02 00:00:00,2500
5,2,2021-04-03 00:00:00,2200


In [ ]:
sql = '''select event_date, user_id, revenue,
round(revenue*1.0/lag(revenue) over(partition by user_id order by event_date),2) as prev_rev
from revenue
order by event_date desc, user_id

'''

In [ ]:
select(sql)

,event_date,user_id,revenue,prev_rev
0,2021-04-03 00:00:00,1,3000,3.75
1,2021-04-03 00:00:00,2,2200,0.88
2,2021-04-02 00:00:00,1,800,0.80
3,2021-04-02 00:00:00,2,2500,1.25
4,2021-04-01 00:00:00,1,1000,NaN
5,2021-04-01 00:00:00,2,2000,NaN


**Second Highest Salary**

In [ ]:
sql = '''Create table If Not Exists Employee (id int, salary int)
insert into Employee (id, salary) values ('1', '100')
insert into Employee (id, salary) values ('2', '200')
insert into Employee (id, salary) values ('3', '300')'''

In [ ]:
sql = ';\n'.join(sql.split('\n'))

In [ ]:
print(sql)

Create table If Not Exists Employee (id int, salary int);
insert into Employee (id, salary) values ('1', '100');
insert into Employee (id, salary) values ('2', '200');
insert into Employee (id, salary) values ('3', '300')


In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select * from Employee'''

In [ ]:
select(sql)

,id,salary
0,1,100
1,2,200
2,3,300


In [ ]:
sql = '''select max(t.salary) as SecondHighestSalary
from Employee t
where t.salary not in (select max(t.salary) as SecondHighestSalary
from Employee t)'''

In [ ]:
select(sql)

,SecondHighestSalary
0,200


**Customers Who Never Order**

In [ ]:
sql = '''Create table If Not Exists Customers (id int, name varchar(255))
Create table If Not Exists Orders (id int, customerId int)
insert into Customers (id, name) values ('1', 'Joe')
insert into Customers (id, name) values ('2', 'Henry')
insert into Customers (id, name) values ('3', 'Sam')
insert into Customers (id, name) values ('4', 'Max')
insert into Orders (id, customerId) values ('1', '3')
insert into Orders (id, customerId) values ('2', '1')'''

In [ ]:
sql = ';\n'.join(sql.split('\n'))

In [ ]:
print(sql)

Create table If Not Exists Customers (id int, name varchar(255));
Create table If Not Exists Orders (id int, customerId int);
insert into Customers (id, name) values ('1', 'Joe');
insert into Customers (id, name) values ('2', 'Henry');
insert into Customers (id, name) values ('3', 'Sam');
insert into Customers (id, name) values ('4', 'Max');
insert into Orders (id, customerId) values ('1', '3');
insert into Orders (id, customerId) values ('2', '1')


In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''SELECT A.Name from Customers A
WHERE A.Id NOT IN (SELECT B.CustomerId from Orders B)
'''

In [ ]:
select(sql)

,name
0,Henry
1,Max


In [ ]:
sql = '''Create table If Not Exists Employee1 (id int, name varchar(255), salary int, departmentId int)
DROP TABLE IF EXISTS Employee1
Create table If Not Exists Department (id int, name varchar(255))
DROP TABLE IF EXISTS Department
insert into Employee1 (id, name, salary, departmentId) values ('1', 'Joe', '70000', '1')
insert into Employee1 (id, name, salary, departmentId) values ('2', 'Jim', '90000', '1')
insert into Employee1 (id, name, salary, departmentId) values ('3', 'Henry', '80000', '2')
insert into Employee1 (id, name, salary, departmentId) values ('4', 'Sam', '60000', '2')
insert into Employee1 (id, name, salary, departmentId) values ('5', 'Max', '90000', '1')
insert into Department (id, name) values ('1', 'IT')
insert into Department (id, name) values ('2', 'Sales')'''

In [ ]:
sql = ';\n'.join(sql.split('\n'))

In [ ]:
print(sql)

Create table If Not Exists Employee1 (id int, name varchar(255), salary int, departmentId int);
DROP TABLE IF EXISTS Employee1;
Create table If Not Exists Department (id int, name varchar(255));
DROP TABLE IF EXISTS Department;
insert into Employee1 (id, name, salary, departmentId) values ('1', 'Joe', '70000', '1');
insert into Employee1 (id, name, salary, departmentId) values ('2', 'Jim', '90000', '1');
insert into Employee1 (id, name, salary, departmentId) values ('3', 'Henry', '80000', '2');
insert into Employee1 (id, name, salary, departmentId) values ('4', 'Sam', '60000', '2');
insert into Employee1 (id, name, salary, departmentId) values ('5', 'Max', '90000', '1');
insert into Department (id, name) values ('1', 'IT');
insert into Department (id, name) values ('2', 'Sales')


In [ ]:
sql ='''select * from Employee1'''



In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'Company':['Brand name', 'Ecco Shoes', 'Adic Babic']})

In [ ]:
df.dtypes

Company    object
dtype: object

In [ ]:
df['short_name'] = df.Company.apply(lambda x: x.split(' ')[-1])

In [ ]:
df

,Company,short_name
0,Brand name,name
1,Ecco Shoes,Shoes
2,Adic Babic,Babic
